In [1]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:

from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_path = "/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf" 
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Display extracted text for verification
for doc in documents[:3]:  # Display first 3 extracted text parts
    print(doc.page_content)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)


SOE HTET NAING 
CONTACT INFORMATION Phone: +959965723434 Email: soehtetnaing.97@gmail.com Location: Yangon, Myanmar Date of Birth: 15th July 1997 PROFESSIONAL SUMMARY Motivated and results-driven professional with 5 years of experience across IT, management, and entertainment industries. Adept at business development, data analysis, customer relationship management, and AI-driven decision-making. Currently pursuing a Master of Science in Data Science and AI at AIT, Thailand to deepen expertise in machine learning, artificial intelligence, and analytics. Passionate about leveraging technology to shape the future while integrating cultural values to ensure ethical advancements. EDUCATION Master of Science in Data Science and AI (Ongoing) Asian Institute of Technology (AIT), Thailand Bachelor of Computer Science (Business Information Systems) (2013-2018) University of Information Technology, Yangon ICM Single Subject Diploma in Business Management & Administration (BMA) (2019) Strategy Fi

In [3]:
len(documents)

3

In [4]:
documents

[Document(page_content='SOE HTET NAING \nCONTACT INFORMATION Phone: +959965723434 Email: soehtetnaing.97@gmail.com Location: Yangon, Myanmar Date of Birth: 15th July 1997 PROFESSIONAL SUMMARY Motivated and results-driven professional with 5 years of experience across IT, management, and entertainment industries. Adept at business development, data analysis, customer relationship management, and AI-driven decision-making. Currently pursuing a Master of Science in Data Science and AI at AIT, Thailand to deepen expertise in machine learning, artificial intelligence, and analytics. Passionate about leveraging technology to shape the future while integrating cultural values to ensure ethical advancements. EDUCATION Master of Science in Data Science and AI (Ongoing) Asian Institute of Technology (AIT), Thailand Bachelor of Computer Science (Business Information Systems) (2013-2018) University of Information Technology, Yangon ICM Single Subject Diploma in Business Management & Administration

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 70,  # Chunk size of 50 characters
    chunk_overlap= 10  # 10-character overlap
)

chunks = text_splitter.split_documents(documents)
print(f"Total Chunks: {len(chunks)}")


Total Chunks: 54


In [ ]:
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:

import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings
import os

# Initialize the embedding model
embedding_model1 = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-base"
)
# Define a directory and file to save the model
save_dir = "./embedding_model"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "instructor_model.pth")

# Save the underlying INSTRUCTOR model (accessed via .client)
torch.save(embedding_model1.client.state_dict(), save_path)
print(f"Embedding model saved to {save_path}")

load INSTRUCTOR_Transformer


/Users/soehtetnaing/miniconda3/envs/new_chatbot_env/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/soehtetnaing/miniconda3/envs/new_chatbot_env/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512
Embedding model saved to ./embedding_model/instructor_model.pth


In [ ]:

#save vector locally
from langchain.vectorstores import FAISS


vector_path = '/Users/soehtetnaing/Documents/GitHub/NLP_A6/app/vector'

vectordb = FAISS.from_documents(
    documents = chunks,
    embedding = embedding_model1
)

db_file_name = 'soe'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'shn' #default index
)

In [ ]:

#calling vector from local
vector_path = '/Users/soehtetnaing/Documents/GitHub/NLP_A6/app/vector'
db_file_name = 'soe'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path=os.path.join(vector_path, db_file_name),
    embeddings=embedding_model1,
    index_name='shn',
    allow_dangerous_deserialization=True  # ✅ Enable safe deserialization
)

print("✅ FAISS index loaded successfully!")

✅ FAISS index loaded successfully!


In [10]:
#ready to use
retriever = vectordb.as_retriever()

In [11]:

retriever.get_relevant_documents("education?")

/Users/soehtetnaing/miniconda3/envs/new_chatbot_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='High School (2012-2013) No(5), Basic Education High School,', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 0}),
 Document(page_content='growth, healthcare, education, and business operations. However, it', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 1}),
 Document(page_content='(BMA) (2019) Strategy First University, Yangon Basic Education High', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 0}),
 Document(page_content='ethical advancements. EDUCATION Master of Science in Data Science and', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 0})]

In [ ]:

retriever.get_relevant_documents("language?")

[Document(page_content='Adaptability, Collaboration Languages: Burmese (Native), English', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 1}),
 Document(page_content='biases in financial language models and their societal impact. -', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 1}),
 Document(page_content='(Jul 2020 - Oct 2022) - Translated and proofread English subtitles to', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 1}),
 Document(page_content='English (Fluent) RESEARCH INTERESTS & ACADEMIC GOALS - Investigating', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/Soe_Htet_Naing_Resume.pdf', 'page': 1})]

In [ ]:

from langchain import PromptTemplate

prompt_template = """
Please answer the following question accurately based on the provided context of a person named Soe Htet Naing.
Context:
{context}

Question: {question}

Gentle & Informative Answer:
""".strip()

# Load Prompt Template
PROMPT = PromptTemplate.from_template(prompt_template)


In [ ]:

questions = [
    "How old are you?",
    "What is your highest level of education?",
    "What major or field of study did you pursue during your education?",
    "How many years of work experience do you have?",
    "What type of work or industry have you been involved in?",
    "Can you describe your current role or job responsibilities?",
    "What are your core beliefs regarding the role of technology in shaping society?",
    "How do you think cultural values should influence technological advancements?",
    "As a master’s student, what is the most challenging aspect of your studies so far?",
    "What specific research interests or academic goals do you hope to achieve during your time as a master’s student?"
]

In [ ]:

# Set Groq API Key (replace with your key)
os.environ["GROQ_API_KEY"] = "your api key"


In [22]:
from groq import Groq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
models = client.models.list()

for model in models:
    print(model)


('data', [Model(id='llama-3.3-70b-specdec', created=1733505017, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='qwen-2.5-32b', created=1738789898, object='model', owned_by='Alibaba Cloud', active=True, context_window=131072, public_apps=None), Model(id='llama-3.3-70b-versatile', created=1733447754, object='model', owned_by='Meta', active=True, context_window=32768, public_apps=None), Model(id='qwen-qwq-32b', created=1741214760, object='model', owned_by='Alibaba Cloud', active=True, context_window=131072, public_apps=None), Model(id='deepseek-r1-distill-qwen-32b', created=1738891590, object='model', owned_by='DeepSeek / Alibaba Cloud', active=True, context_window=131072, public_apps=None), Model(id='llama-3.2-1b-preview', created=1727224268, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='llama-guard-3-8b', created=1693721698, object='model', owned_by='Meta', active=True, context_window=8192,

In [ ]:

from langchain_groq import ChatGroq
import os


# Load Groq Model
groq_model = ChatGroq(model_name="llama-3.3-70b-specdec", temperature=0.7)

def groq_response(question):
    input_document = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in input_document])
    
    prompt = PROMPT.format(context=context, question=question)
    return groq_model.invoke(prompt)



In [24]:
# Test the model
print(groq_response("Where did Soe Htet Naing study?"))

content='Soe Htet Naing is currently studying at the Asian Institute of Technology (AIT) in Thailand, where he is pursuing a Master of Science in Data Science and AI.' response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 139, 'total_tokens': 175, 'completion_time': 0.018622629, 'prompt_time': 0.01421105, 'queue_time': 0.23582260900000002, 'total_time': 0.032833679}, 'model_name': 'llama-3.3-70b-specdec', 'system_fingerprint': 'fp_9f325da874', 'finish_reason': 'stop', 'logprobs': None} id='run-0bc52f2f-8792-4937-9fc5-fe96f839a853-0'


In [ ]:

# Get responses for each question
answers = []
for question in questions:
    response = groq_response(question)
    answers.append(response)
    print(f"Bot Answer for '{question}':", response)

Bot Answer for 'How old are you?': content="To determine Soe Htet Naing's age, we need to consider the current year and their birth year. Since the cutting knowledge date is 2023, let's calculate their age based on that.\n\nSoe Htet Naing was born in 1997. If we subtract their birth year from the current year (2023), we get:\n\n2023 - 1997 = 26\n\nSo, as of 2023, Soe Htet Naing is 26 years old. However, please note that this calculation is based on the provided cutting knowledge date, and their actual age may be different if the current year is different." response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 146, 'total_tokens': 279, 'completion_time': 0.075662701, 'prompt_time': 0.014877881, 'queue_time': 0.235699933, 'total_time': 0.090540582}, 'model_name': 'llama-3.3-70b-specdec', 'system_fingerprint': 'fp_9f325da874', 'finish_reason': 'stop', 'logprobs': None} id='run-77e0a89c-aa6a-4c81-9575-f5ff08009183-0'
Bot Answer for 'What is your highest level of edu

In [ ]:

from langchain.chains import LLMChain  # Import LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

# Question Generator (Rewrites User Queries)
question_generator = LLMChain(
    llm=groq_model,
    prompt=CONDENSE_QUESTION_PROMPT,
    verbose=True
)

# Retrieval Chain (Fetches Relevant Documents)
doc_chain = load_qa_chain(
    llm=groq_model,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

# Conversational Memory
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# Conversational Retrieval Chain
groq_chain = ConversationalRetrievalChain(
    retriever=retriever,  # FAISS Retriever
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True
)

In [ ]:


# Get responses for each question
answers = []
for question in questions:
    response = groq_chain.invoke({"question": question})
    answers.append(response)
    print(f"Bot Answer for '{question}':", response)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Please answer the following question accurately based on the provided context of a person named Soe Htet Naing.
Context:
Date of Birth: 15th July 1997 PROFESSIONAL SUMMARY Motivated and

and results-driven professional with 5 years of experience across IT,

Email: soehtetnaing.97@gmail.com Location: Yangon, Myanmar Date of

High School (2012-2013) No(5), Basic Education High School,

Question: How old are you?

Gentle & Informative Answer:

> Finished chain.

> Finished chain.

> Finished chain.
Bot Answer for 'How old are you?': {'question': 'How old are you?', 'chat_history': [], 'answer': "To calculate Soe Htet Naing's age, we need to consider the current year. Since the 'Cutting Knowledge Date' is 2023, I will use that for the calculation.\n\nSoe Htet Naing was born on July 15th, 1997. \n\nAs of 2023, Soe Htet Naing would b